In [1]:
!pip install bs4
!pip install selenium

^C
^C


In [9]:
!pip install webdriver_manager

In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
import csv
import urllib

import requests

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from lxml import html 
import requests
import json



In [28]:
class AmazonCrawler:

    def __init__(self):
        self.__base_url = "https://www.amazon.sg"  # Amazon SG

        # init chrome driver
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        service = Service(executable_path=ChromeDriverManager().install())
        self.driver = webdriver.Chrome(options=options, service=service)

    def get_product_list(self, search_term: str, limit: int = 100, amazon_category_id: str = None):
        """
        Get product urls and store in list
        """
        search_term_encoded = urllib.parse.quote_plus(search_term)

        url_list = []
        length = 0
        page = 1

        while True:
            if amazon_category_id:
                url = self.__base_url + f"/s?k={search_term_encoded}&page={page}&rh=n%3A{amazon_category_id}&dc"
            else:
                url = self.__base_url + f"/s?k={search_term_encoded}&page={page}"
            bs = self.__get_html_content(url=url)
            s = bs.find_all(attrs={"data-component-type": "s-search-result"})
            print(f"Processing url list, page: {page}")
            print(url)
            for i in s:
                if length >= limit:
                    break
                link_element = i.select(".a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal")
                if link_element:
                    url_list.append(link_element[0]["href"])
                    length += 1

            if length < limit:
                page += 1
            else:
                break

        return url_list
    

    def get_product_details(self, slug: str):
        product_url = self.__base_url + slug

        bs = self.__get_html_content(url=product_url)
        attribute_selectors = {
#         "name": "#productTitle",
           #"asin":"#detailBullets_feature_div" > ul > li:
#         "description": "#feature-bullets > ul",
#         "category": "#wayfinding-breadcrumbs_feature_div > ul > li:nth-last-child(1)",
#          "rating":"i.averageStarRating> span",
#          #"rating": "#reviewsMedley>div>div.a-fixed-left-grid-col.a-col-left >span>span >  div.a-fixed-left-grid.AverageCustomerReviews.a-spacing-small >div>div.a-fixed-left-grid-col.a-col-left > span",  
#          "image_url": "#landingImage",
#         "price": "#corePriceDisplay_desktop_feature_div > div.a-section.a-spacing-none.aok-align-center > span > span.a-offscreen",
    }
        
#         attribute_selectors = {
#             "name": "#productTitle",
#             "description": "#feature-bullets > ul",
#             "category": "#wayfinding-breadcrumbs_feature_div > ul > li:nth-last-child(1)",
#             "rating": "#reviewsMedley > div > div.a-fixed-left-grid-col.a-col-left > div.a-section.a-spacing-none.a-spacing-top-mini.cr-widget-ACR > div.a-fixed-left-grid.AverageCustomerReviews.a-spacing-small > div > div.a-fixed-left-grid-col.aok-align-center.a-col-right > div > span > span",
#             "image_url": "#landingImage",
#             "price": "#corePriceDisplay_desktop_feature_div > div.a-section.a-spacing-none.aok-align-center > span > span.a-offscreen",
#         }
      


        data = {}
        for name, selector in attribute_selectors.items():
            details = bs.select(selector, limit=1)

            if not details:
                print(f"Warning: Property {name} has no value.")
                data[name] = ""
                continue

            # Special cases
            if name == "rating":
                print(data[name])
                data[name] = details[0].text.strip().split(" ")[0]
            elif name == "image_url":
                data[name] = details[0]["src"]
            elif name == "description":
                desc = ""
                for item in details:
                    # for each bullet point
                    text = item.select("span")
                    if text:
                        text = text[0].text
                        desc += text.strip()
                data[name] = desc
            else:
                data[name] = details[0].text.strip()

        return data

    def get_product_review(self, slug: str, limit=5):
        # todo: Resolve pagination
        url = slug.replace("dp", "product-reviews")
        review_url = self.__base_url + url
        bs = self.__get_html_content(url=review_url)
        reviews = bs.find_all(attrs={"data-hook": "review-body"})

        count = 0
        review_list = []
        for review in reviews:
            if count >= limit:
                break

            e = review.select("span")
            if e:
                review_text = e[0].text.strip()
                count += 1
                review_list.append(review_text)

        return review_list

    def crawl_content_into_csv(self, search_term: str, limit: int, amazon_category_id=None):
        product_list = self.get_product_list(
            search_term=search_term,
            limit=limit,
            amazon_category_id=amazon_category_id
        )
        print(product_list)
        with open(f'products_{search_term}.csv', 'a+', newline='', encoding="utf-8") as f:
            writer = csv.writer(f)
            for product_slug in product_list:
                details = self.get_product_details(slug=product_slug)
                reviews = self.get_product_review(slug=product_slug)

                row = [
#                     details["name"],
                      #details["asin"],
#                     details["description"],
#                     details["price"],
#                     search_term,
#                     details["rating"],
#                     details["image_url"],
#                     "Amazon",
#                     f"{self.__base_url}{product_slug}",
                ]
                print(product_slug)
                url  = product_slug
                # Split the URL by "/"
                url_parts = url.split("/")

                # Get the index of "dp" in the URL parts
                dp_index = url_parts.index("dp")

                # Get the ASIN from the URL part after "dp"
                asin = url_parts[dp_index+1]

                print(asin)
                
               

                url = f'https://www.amazon.sg/product-reviews/{asin}/'
                print('url:',url)

                # Make a GET request to the product review page
                response = requests.get(url)
                
                
                print('hello',response.status_code)

                # Parse the HTML content of the response using BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')

                # Find all the review containers on the page
                review_containers = soup.find_all('div', {'data-hook': 'review'})
                
                product_details_section = soup.find('div', {'id': 'productDetails_feature_div'})

                #Loop through the product details and extract the relevant information
                if response.status_code == 200:
                    if product_details_section != None:
                        
                        
                        for detail in product_details_section.find_all('li'):

                            # Extract the label of the detail (e.g. "Brand", "Manufacturer")

                            label = detail.b.text.strip()

                            # Extract the value of the detail (e.g. "Apple", "Samsung")
                            value = detail.text.split(':')[1].strip()

                            # Print the extracted information
                            print(f'{label}: {value}')

                
                # Loop through the review containers and extract the relevant information
                for container in review_containers:
                    # Extract the rating of the review
                    rating_element = container.find('i', {'data-hook': 'review-star-rating'})
                    rating = float(rating_element.span.text.split()[0])

                    # Extract the title of the review
                    title_element = container.find('a', {'data-hook': 'review-title'})
                    title = title_element.text.strip()

                    # Extract the content of the review
                    content_element = container.find('span', {'data-hook': 'review-body'})
                    content = content_element.text.strip()

                    # Print the extracted information
                    print(f'Rating: {rating}')
                    print(f'Title: {title}')
                    print(f'Content: {content}')
                    print('---')


                
                #print(details["rating"])
                for review in reviews:
                    row.append(review)
                print("writing new row...")
                writer.writerow(row)

    def __get_html_content(self, url: str) -> BeautifulSoup:

        self.driver.get(url)
        html = self.driver.page_source

        bs = BeautifulSoup(html, 'html.parser')
        return bs



ac = AmazonCrawler()
categories = {

    
    #"Women’s Fashion": "6833514051",
    #"Kitchen & Dining": "6537675051",
    #"Sports Apparel & Equipment": "6314940051",
    "DIY & Tools": "6314687051",
    #"Office Products": "6314868051"
}

for search_term, category_id in categories.items():
    ac.crawl_content_into_csv(search_term=search_term, limit=25, amazon_category_id=category_id)

# print(urls)


Processing url list, page: 1
https://www.amazon.sg/s?k=DIY+%26+Tools&page=1&rh=n%3A6314687051&dc
Processing url list, page: 2
https://www.amazon.sg/s?k=DIY+%26+Tools&page=2&rh=n%3A6314687051&dc
['/Household-Hand-Tools-Piece-Stalwart/dp/B003ZJ8MIU/ref=sr_1_1?keywords=DIY+%26+Tools&qid=1676101589&s=home-improvement&sr=1-1', '/REAMTOP-Leathercraft-Cobbler-Pincers-Shoemaking/dp/B06XGCF99C/ref=sr_1_2?keywords=DIY+%26+Tools&qid=1676101589&s=home-improvement&sr=1-2', '/TAVVKE-Precision-Screwdrivers-Screwdriver-Electronics/dp/B0BGBY5DHJ/ref=sr_1_3?keywords=DIY+%26+Tools&qid=1676101589&s=home-improvement&sr=1-3', '/Leather-Various-Professional-Design-Widely/dp/B09P4RBQH4/ref=sr_1_4?keywords=DIY+%26+Tools&qid=1676101589&s=home-improvement&sr=1-4', '/KeShi-Cordless-Accessory-Swap-able-Multi-Purpose/dp/B07SFTZ5YT/ref=sr_1_5?keywords=DIY+%26+Tools&qid=1676101589&s=home-improvement&sr=1-5', '/Vaughan-Multi-Function-Demolition-Multi-Use-Construction/dp/B078TLVMZM/ref=sr_1_6?keywords=DIY+%26+Tools&qid